# Test Full Code with Birmingham
### Gavin Rolls

This notebook will be a compilation of the methods I've applied to London as my area of study and attempt to recreate the model in Birmingham

# Data Collection

### Imports + Database Configuration

In [23]:
#Library Imports - using DuckDB for Overture Import

#Database
import duckdb

#Basics
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim

#OpenStreetMap
import osmnx as ox

#Shapely
from geopy.geocoders import Nominatim
from shapely.geometry import box
from shapely import wkt
import shapely.geometry
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry import mapping
from shapely.geometry import shape

#Plots
import matplotlib.pyplot as plt

#Warning Supression
import warnings

In [3]:
#Config SQL
%pip install ipython-sql duckdb duckdb-engine jupysql --quiet
%pip install --upgrade grpcio --quiet
%load_ext sql

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [6]:
%%sql      
INSTALL httpfs;

LOAD httpfs;

INSTALL spatial;

LOAD spatial;

SET s3_region='us-west-2';

Empty DataFrame
Columns: [Success]
Index: []

### Download Buildings - Overture

Downloaded as 'data/bham_buildings_overture.geojson'

In [12]:
#Get Birmingham Bounding Box

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapi")

# Get location data
location = geolocator.geocode("Birmingham")

# Get the bounding box
bounding_box = location.raw['boundingbox']

# Convert bounding box to coordinates
min_lat, max_lat = float(bounding_box[0]), float(bounding_box[1])
min_lon, max_lon = float(bounding_box[2]), float(bounding_box[3])

print(min_lon)
print(min_lat)
print(max_lon)
print(max_lat)

-2.0336486
52.381053
-1.7288417
52.6087058


In [14]:
%%sql

LOAD azure;

SET azure_storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=overturemapswestus2;AccountKey=;EndpointSuffix=core.windows.net';
COPY (
SELECT
    names.primary as primary_name,
    height,
    level,
    ST_GeomFromWKB(geometry) as geometry
FROM read_parquet('azure://release/2024-05-16-beta.0/theme=buildings/type=building/*', filename=true, hive_partitioning=1)
WHERE primary_name IS NOT NULL
AND bbox.xmin > -2.0336486
AND bbox.xmax < -1.7288417
AND bbox.ymin > 52.381053
AND bbox.ymax < 52.6087058
) TO 'data/overture_data/bham_buildings_overture.geojson'
WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Empty DataFrame
Columns: [Success]
Index: []

### Download POIs - Overture

Downloaded as 'data/bham_places_overture.geojson''

In [15]:
%%sql

COPY (
    SELECT
        names.primary AS name,
        categories.main as category,
        ROUND(confidence,2) as confidence,
        ST_GeomFromWKB(geometry) as geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=places/*/*')
WHERE
    bbox.xmin BETWEEN -2.0336486 AND -1.7288417 AND
    bbox.ymin BETWEEN 52.381053 AND 52.6087058
) TO 'data/overture_data/bham_places_overture.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Empty DataFrame
Columns: [Success]
Index: []

### Overture Download Stats

In [35]:
#Basic overview stats of Birmingham Data - Using the locally saved files here
buildings = gpd.read_file('data/overture_data/bham_buildings_overture.geojson')
places = gpd.read_file('data/overture_data/bham_places_overture.geojson')

#Count of Features
print("Birmingham Building Count: " + str(buildings.shape[0]))
print("Birmingham POI count: " + str(places.shape[0]))

Birmingham Building Count: 11987
Birmingham POI count: 46781


### Download Buildings - OSM

Downloaded as 'data/osmbuildings/building_footprints.geojson'

In [36]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

#Test new config
ox.config(use_cache=True, log_console=True)

buildings = ox.features_from_place(place_name, tags={'building': True})
buildings = buildings[buildings.geometry.notnull()]
building_footprints = buildings[buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in building_footprints.columns:
    if building_footprints.apply(lambda x: isinstance(x, list)).any():
        building_footprints = building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

building_footprints = building_footprints[['name', 'geometry']].reset_index()
print(building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
building_footprints.to_file("data/osm_data/bham_buildings.geojson", driver="GeoJSON")

/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_92764/2788206819.py:5: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


       element_type     osmid                                          name  \
0               way  16966777                                           NaN   
1               way  16966784                                           NaN   
2               way  16966785                                           NaN   
3               way  16966788                                           NaN   
4               way  16966789                                           NaN   
...             ...       ...                                           ...   
256261     relation  15607352                   Beauchamp Convenience Store   
256262     relation  15809922                                       Block A   
256263     relation  15823526                                           NaN   
256264     relation  16414329  Birmingham Settlement - 610 Community Centre   
256265     relation  16542773                                           NaN   

                                                 ge

### Extract Building Type Info from OSM - Commercial

In [37]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

commercial_buildings = ox.features_from_place(place_name, tags={'building': ['commercial']})

commercial_buildings = commercial_buildings[commercial_buildings.geometry.notnull()]
commercial_building_footprints = commercial_buildings[commercial_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in commercial_building_footprints.columns:
    if commercial_building_footprints.apply(lambda x: isinstance(x, list)).any():
        commercial_building_footprints = commercial_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

commercial_building_footprints = commercial_building_footprints[['name', 'geometry']].reset_index()
print(commercial_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
commercial_building_footprints.to_file("data/osm_data/bham_commercial_buildings.geojson", driver="GeoJSON")

    element_type       osmid                     name  \
0            way    17405658                      NaN   
1            way    20002347  Dental Care Partnership   
2            way    22763584            Station House   
3            way    22763760                      NaN   
4            way    22763952                      NaN   
..           ...         ...                      ...   
312          way  1259115143                      NaN   
313          way  1259115144                      NaN   
314          way  1259115145                      NaN   
315     relation      333915      The Custard Factory   
316     relation     7076030                Eagle Two   

                                              geometry  
0    POLYGON ((-1.82146 52.56186, -1.82179 52.56209...  
1    POLYGON ((-1.82680 52.55679, -1.82680 52.55680...  
2    POLYGON ((-1.82027 52.56533, -1.82036 52.56528...  
3    POLYGON ((-1.81624 52.56247, -1.81633 52.56237...  
4    POLYGON ((-1.82061 52.565

### Extract Building Type Info from OSM - Office

In [39]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

ox.config(use_cache=True, log_console=True)

office_buildings = ox.features_from_place(place_name, tags={'building': ['office']})

office_buildings = office_buildings[office_buildings.geometry.notnull()]
office_building_footprints = office_buildings[office_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in office_building_footprints.columns:
    if office_building_footprints.apply(lambda x: isinstance(x, list)).any():
        office_building_footprints[col] = office_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

office_building_footprints = office_building_footprints[['name', 'geometry']].reset_index()
print(office_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
office_building_footprints.to_file("data/osm_data/bham_office_buildings.geojson", driver="GeoJSON")


    element_type     osmid                     name  \
0            way  22820818        Baskerville House   
1            way  28292471     No. 1 Colmore Square   
2            way  28425056         McLaren Building   
3            way  28455010        Gazette Buildings   
4            way  28514237                      NaN   
..           ...       ...                      ...   
190     relation   1298933           Fountain Court   
191     relation   3417218     Eight Brindley Place   
192     relation   3417219     Seven Brindley Place   
193     relation   3417220       Ten Brindley Place   
194     relation   4631334  Birmingham Royal Ballet   

                                              geometry  
0    POLYGON ((-1.90809 52.48018, -1.90735 52.48039...  
1    POLYGON ((-1.89668 52.48235, -1.89675 52.48258...  
2    POLYGON ((-1.89265 52.48192, -1.89247 52.48183...  
3    POLYGON ((-1.89313 52.48261, -1.89339 52.48236...  
4    POLYGON ((-1.90172 52.47568, -1.90125 52.47528...

/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_92764/2941556620.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


### Extract Building Type Info from OSM - Residential

In [40]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

ox.config(use_cache=True, log_console=True)

residential_buildings = ox.features_from_place(place_name, tags={'building': ['residential']})

residential_buildings = residential_buildings[residential_buildings.geometry.notnull()]
residential_building_footprints = residential_buildings[residential_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in residential_building_footprints.columns:
    if residential_building_footprints.apply(lambda x: isinstance(x, list)).any():
        residential_building_footprints[col] = residential_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

residential_building_footprints = residential_building_footprints[['name', 'geometry']].reset_index()
print(residential_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
residential_building_footprints.to_file("data/osm_data/bham_residential_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_92764/2012114761.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


       element_type     osmid           name  \
0               way  16966777            NaN   
1               way  16966784            NaN   
2               way  16966785            NaN   
3               way  16966788            NaN   
4               way  16966789            NaN   
...             ...       ...            ...   
189381     relation  10407216            NaN   
189382     relation  11382596  Bentley House   
189383     relation  12197160            NaN   
189384     relation  15809922        Block A   
189385     relation  16542773            NaN   

                                                 geometry  
0       POLYGON ((-1.82461 52.55549, -1.82450 52.55543...  
1       POLYGON ((-1.82324 52.55438, -1.82301 52.55434...  
2       POLYGON ((-1.82273 52.55403, -1.82264 52.55400...  
3       POLYGON ((-1.82341 52.55414, -1.82338 52.55412...  
4       POLYGON ((-1.82311 52.55396, -1.82302 52.55391...  
...                                                   ...  
189

### Extract Building Type Info from OSM - Retail

In [41]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

ox.config(use_cache=True, log_console=True)

retail_buildings = ox.features_from_place(place_name, tags={'building': ['retail']})

retail_buildings = retail_buildings[retail_buildings.geometry.notnull()]
retail_building_footprints = retail_buildings[retail_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in retail_building_footprints.columns:
    if retail_building_footprints.apply(lambda x: isinstance(x, list)).any():
        retail_building_footprints[col] = retail_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

retail_building_footprints = retail_building_footprints[['name', 'geometry']].reset_index()
print(retail_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
retail_building_footprints.to_file("data/osm_data/bham_retail_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_92764/291704794.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


     element_type     osmid                         name  \
0             way  16969083                        Boots   
1             way  17273169                          NaN   
2             way  20002348                  Vesey Manor   
3             way  20002352             Driffold Gallery   
4             way  23585174                     One Stop   
...           ...       ...                          ...   
3499     relation   2912290                  Sainsbury's   
3500     relation   5536524                Grand Central   
3501     relation  10748407             Picadilly Arcade   
3502     relation  14397641                          NaN   
3503     relation  15607352  Beauchamp Convenience Store   

                                               geometry  
0     POLYGON ((-1.82369 52.55768, -1.82379 52.55771...  
1     POLYGON ((-1.89325 52.47748, -1.89313 52.47745...  
2     POLYGON ((-1.82665 52.55648, -1.82665 52.55630...  
3     POLYGON ((-1.82722 52.55565, -1.82734 52.

# Data Cleaning
### Taken from v2

## Data Loading + Cleaning

### UK BRES Employment Data

In [53]:
#Skip the first six rows because they're header information
empl_data = pd.read_csv('data/employment_data/lsoa_by_industry.csv', skiprows=7, delimiter=',')

unnamed_cols = empl_data.columns[empl_data.columns.str.contains('^Unnamed:')]
empl_data.drop(columns=unnamed_cols, inplace=True)

#Separate name into LSOA11CD and LSOA11NM
def split_column(value):
    if isinstance(value, str) and 'lsoa2011:' in value:
        parts = value.split('lsoa2011:')[1]
        code, name = parts.split(' : ')
        return code.strip(), name.strip()
    else:
        return None, None

empl_data[['LSOA11CD', 'LSOA11NM']] = empl_data['Area'].apply(lambda x: pd.Series(split_column(x)))

# Drop rows not in Birmingham
empl_data.dropna(subset=['LSOA11NM'], inplace=True)
empl_data = empl_data[empl_data['LSOA11NM'].str.contains('Birmingham')]

print("Num Rows (LSOAs) Before Cleaning: " + str(empl_data.shape[0]))

#There appear to be a bunch of duplicates so I'm going to get rid of them now
empl_data.drop_duplicates(inplace=True)

print("Num Rows (LSOAs) After Cleaning: " + str(empl_data.shape[0]))

Num Rows (LSOAs) Before Cleaning: 639
Num Rows (LSOAs) After Cleaning: 639


In [55]:
#Create Total Employment Column
empl_data['total_employment'] = empl_data[empl_data.columns[1:-2]].sum(axis=1)

empl_data.head()

Area  \
13485  lsoa2011:E01008881 : Birmingham 067A   
13486  lsoa2011:E01008882 : Birmingham 066A   
13487  lsoa2011:E01008883 : Birmingham 078A   
13488  lsoa2011:E01008884 : Birmingham 078B   
13489  lsoa2011:E01008885 : Birmingham 076A   

       01 : Crop and animal production, hunting and related service activities  \
13485                                                0.0                         
13486                                                0.0                         
13487                                                0.0                         
13488                                                0.0                         
13489                                                0.0                         

       02 : Forestry and logging  03 : Fishing and aquaculture  \
13485                        0.0                           0.0   
13486                        0.0                           0.0   
13487                        0.0                           0.0   
13488                        0.0                           0.0   
13489                        0.0                           0.0   

       05 : Mining of coal and lignite  \
13485                              0.0   
13486                              0.0   
13487                              0.0   
13488                              0.0   
13489                              0.0   

       06 : Extraction of crude petroleum and natural gas  \
13485                                                0.0    
13486                                                0.0    
13487                                                0.0    
13488                                                0.0    
13489                                                0.0    

       07 : Mining of metal ores  08 : Other mining and quarrying  \
13485                        0.0                              0.0   
13486                        0.0                              0.0   
13487                        0.0                              0.0   
13488                        0.0                              0.0   
13489                        0.0                              0.0   

       09 : Mining support service activities  \
13485                                     0.0   
13486                                     0.0   
13487                                     0.0   
13488                                     0.0   
13489                                     0.0   

       10 : Manufacture of food products  ...  \
13485                                0.0  ...   
13486                                0.0  ...   
13487                              150.0  ...   
13488                               30.0  ...   
13489                                0.0  ...   

       93 : Sports activities and amusement and recreation activities  \
13485                                                0.0                
13486                                               10.0                
13487                                                0.0                
13488                                                0.0                
13489                                                0.0                

       94 : Activities of membership organisations  \
13485                                          0.0   
13486                                          0.0   
13487                                         20.0   
13488                                          0.0   
13489                                          0.0   

       95 : Repair of computers and personal and household goods  \
13485                                                0.0           
13486                                                0.0           
13487                                                0.0           
13488                                                5.0           
13489                                                0.0           

       96 : Other personal service activities  \
13485              

### LSOA Geographic Data

In [ ]:
#Get LSOA Shapefile Data
lsoa_geo = gpd.read_file('data/lsoa_data/LSOA_2011_London_gen_MHW.shp')

#Convert to WGS for consistency
lsoa_geo = lsoa_geo.to_crs(epsg=4326)
lsoa_geo = lsoa_geo.drop(lsoa_geo.columns[list(range(3, 8))], axis = 1)

print("Num Rows (LSOAs): " + str(lsoa_geo.shape[0]))

#Get rid of columns I'm not using for now
lsoa_geo = lsoa_geo.drop(columns=['MSOA11CD', 'HHOLDRES', 'COMESTRES', 'POPDEN', 'HHOLDS', 'AVHHOLDSZ'])
lsoa_geo = lsoa_geo.rename(columns={'USUALRES': 'population'})

lsoa_geo.head()